In [3]:
import requests, time
import pandas as pd
from datetime import datetime

In [4]:
BASE_URL = 'https://api.spacetraders.io/v2'

In [5]:
# Get account token
ACCOUNT_TOKEN = None
with open('./token.txt', 'r') as ifile:
    ACCOUNT_TOKEN = ifile.read()

# Get agent token
AGENT_TOKEN = None
with open('./agent_token.txt', 'r') as ifile:
    AGENT_TOKEN = ifile.read()

In [6]:
def get_auth_header():
    return {'Authorization': f'Bearer {ACCOUNT_TOKEN}', 'Content-Type': 'application/json'}

def get_agent_header():
    return {'Authorization': f'Bearer {AGENT_TOKEN}', 'Content-Type': 'application/json'}

def post_request(url, data=None, headers=None):
    """ Makes a POST request to the SpaceTraders API. """
    headers = headers or get_agent_header()
    data = data or dict()
    r = requests.post(url=url, headers=headers, json=data)
    return r

def get_request(url, params=None, headers=None):
    """ Makes a GET request to the SpaceTraders API. """
    headers = headers or get_agent_header()
    r = requests.get(url=url, headers=headers, params=params)
    return r


In [19]:
# Registering a new agent
url = 'https://api.spacetraders.io/v2/register'
callsign = 'RYVIOS'
faction = 'COSMIC'
r = post_request(url, {'symbol': callsign, 'faction': faction})



In [5]:
# Agent info
url = 'https://api.spacetraders.io/v2/my/agent'
r = get_request(url)
r.json()

{'data': {'accountId': 'cmhhh9gk1002vtm16u5rl9pf0',
  'symbol': 'RYVIOS',
  'headquarters': 'X1-TP30-A1',
  'credits': 175000,
  'startingFaction': 'COSMIC',
  'shipCount': 2}}

In [7]:
def get_waypoints_in_system(system, type=None, traits=None):
    """ Returns info on all waypoints in given system. May show waypoints as 'Uncharted'. 
        Parameters:
            - types [str]  : filters type of waypoint
            - traits [str] : filters waypoints with given traits
    
    """
    url = BASE_URL + f'/systems/{system}/waypoints'
    filter_params = dict()
    if type is not None:
        filter_params['type'] = type
    if traits is not None:
        filter_params['traits'] = traits

    nextpage = 1
    data = list()
    total = -1
    while total < 0 or len(data) < total:
        r = requests.get(url, params={'page': nextpage, **filter_params})
        if r.status_code == 200:
            data.extend(r.json()['data'])
            nextpage += 1
            total = r.json()['meta']['total']
        else:
            print(f'[WARNING] Failed to fetch page {nextpage} of waypoints in system {system}.')
            print(f'   [INFO]', r.json())
            return data
        
    return data

In [12]:
r = get_request(BASE_URL + '/systems/X1-TP30/waypoints/X1-TP30-H52/shipyard')

In [13]:
r.json()

{'data': {'symbol': 'X1-TP30-H52',
  'shipTypes': [{'type': 'SHIP_MINING_DRONE'}, {'type': 'SHIP_SURVEYOR'}],
  'modificationsFee': 100,
  'transactions': [],
  'ships': [{'type': 'SHIP_MINING_DRONE',
    'name': 'Mining Drone',
    'description': 'A small, unmanned spacecraft that can be used for mining operations, such as extracting valuable minerals from asteroids.',
    'supply': 'HIGH',
    'purchasePrice': 41604,
    'frame': {'symbol': 'FRAME_DRONE',
     'name': 'Drone',
     'condition': 1,
     'integrity': 1,
     'description': 'A small, unmanned spacecraft used for various tasks, such as surveillance, transportation, or combat.',
     'moduleSlots': 3,
     'mountingPoints': 2,
     'fuelCapacity': 80,
     'requirements': {'power': 1, 'crew': -4},
     'quality': 1},
    'reactor': {'symbol': 'REACTOR_CHEMICAL_I',
     'name': 'Chemical Reactor I',
     'condition': 1,
     'integrity': 1,
     'description': 'A basic chemical power reactor, used to generate electricity f

In [14]:
r = post_request(BASE_URL + '/my/ships', data={'shipType': 'SHIP_MINING_DRONE', 'waypointSymbol': 'X1-TP30-H52'})

In [15]:
r.json()

{'data': {'ship': {'symbol': 'RYVIOS-3',
   'registration': {'name': 'RYVIOS-3',
    'factionSymbol': 'COSMIC',
    'role': 'EXCAVATOR'},
   'nav': {'systemSymbol': 'X1-TP30',
    'waypointSymbol': 'X1-TP30-H52',
    'route': {'destination': {'symbol': 'X1-TP30-H52',
      'type': 'MOON',
      'systemSymbol': 'X1-TP30',
      'x': 40,
      'y': -19},
     'origin': {'symbol': 'X1-TP30-H52',
      'type': 'MOON',
      'systemSymbol': 'X1-TP30',
      'x': 40,
      'y': -19},
     'departureTime': '2025-11-02T13:09:28.803Z',
     'arrival': '2025-11-02T13:09:28.803Z'},
    'status': 'DOCKED',
    'flightMode': 'CRUISE'},
   'crew': {'current': 0,
    'required': 0,
    'capacity': 0,
    'rotation': 'STRICT',
    'morale': 100,
    'wages': 0},
   'frame': {'symbol': 'FRAME_DRONE',
    'name': 'Drone',
    'condition': 1,
    'integrity': 1,
    'description': 'A small, unmanned spacecraft used for various tasks, such as surveillance, transportation, or combat.',
    'moduleSlots': 3

In [20]:
def get_contracts():
    """ Returns list of open contracts. """
    # TODO Account for pagination

    r = get_request(BASE_URL + f'/my/contracts')
    if r.status_code != 200:
        print(f'[ERROR] Failed to fetch contracts.')
        print(f' [INFO]', r.json())
        return list()
    
    return r.json()['data']

In [27]:
get_contracts()

[{'id': 'cmhhq5hargq8wui6x9829gze4',
  'factionSymbol': 'COSMIC',
  'type': 'PROCUREMENT',
  'terms': {'deadline': '2025-11-09T13:06:00.526Z',
   'payment': {'onAccepted': 1515, 'onFulfilled': 7070},
   'deliver': [{'tradeSymbol': 'IRON_ORE',
     'destinationSymbol': 'X1-TP30-H51',
     'unitsRequired': 50,
     'unitsFulfilled': 6}]},
  'accepted': True,
  'fulfilled': False,
  'expiration': '2025-11-03T13:06:00.526Z',
  'deadlineToAccept': '2025-11-03T13:06:00.526Z'}]

In [28]:
def deliver_cargo(contract_id, ship, good, verbose=True):
    """ Delivers the specified good for the contract. Delivers entire inventory if possible.
        Returns status [boolean] - True if delivery successful.
    """

    # Determine units to deliver
    # Units in inventory
    cargo = get_ship_cargo(ship)['inventory']
    in_hold = list(filter(lambda i : i['symbol'] == good))
    if len(in_hold) == 0:
        print(f'[ERROR] Ship {ship} has no {good} to deliver. Delivery aborted.')
        return False
    else:
        in_hold = in_hold[0]['units']

    # Units needed by contract
    contract_r = get_request(BASE_URL + f'/my/contracts/{contract_id}')
    if contract_r.status_code != 200:
        print(f'[ERROR] Failed to fetch contract {contract_id}. Ship {ship} is aborting delivery.')
        return False
    
    delivery = contract_r['data']['deliver']
    required = delivery['unitsRequired'] - delivery['unitsFulfilled']
    to_deliver = min(in_hold, required)

    # Attempt delivery
    r = post_request(BASE_URL + f'/my/contracts/{contract_id}/deliver', data={'shipSymbol': ship, 'tradeSymbol': good, 'units': to_deliver})
    if r.status_code != 200:
        print(f'[ERROR] Ship {ship} failed to deliver ({to_deliver}) {good}. Aborting delivery.')
        print(f' [INFO]', r.json())
        return False
    elif verbose:
        print(f'[INFO] Ship {ship} delivered {to_deliver} {good} for contract {contract_id}.')
        if in_hold >= required:
            print(f'[INFO] Ship {ship} completed contract {contract_id}.')
    return True